In [1]:
import math
import random
import copy
import numpy as np
import pandas as pd

# =============================================================================
# --- CHARGEMENT ET FONCTIONS DE BASE (INCHANGÉES) ---
# =============================================================================
def charger_donnees_depuis_excel(chemin_fichier):
    print(f"--- Chargement des données depuis '{chemin_fichier}' ---")
    donnees = {}
    CONVERSION_MILE_TO_M = 1609.34
    df_puits = pd.read_excel(chemin_fichier, sheet_name='Puits')
    puits_data = {}
    pipe_costs_par_m = {4: round(358.69 / CONVERSION_MILE_TO_M, 5), 6: round(454.06 / CONVERSION_MILE_TO_M, 5)}
    for index, row in df_puits.iterrows():
        nom_puits, diam, debit_str = row['NomPuits'], row['DiametreFlowline_in'], row['Debit_kSm3_day']
        if diam not in pipe_costs_par_m: raise ValueError(f"Coût non défini pour le diamètre {diam}in.")
        puits_data[nom_puits] = {'x': float(row['X']), 'y': float(row['Y']), 'production_sm3d': float(str(debit_str).replace(',', '.')) * 1000, 'diametre_flowline_in': diam, 'CDF_i_dollar_par_m': pipe_costs_par_m[diam]}
    donnees['puits'] = puits_data
    print(f"Données des puits chargées: {len(puits_data)} puits.")
    df_manifolds = pd.read_excel(chemin_fichier, sheet_name='TypesManifold')
    donnees['manifolds'] = {row['TypeID']: {'capacite_puits': int(row['CapacitePuits']), 'cout_installation_dollar': float(row['CoutInstallation_dollar'])} for index, row in df_manifolds.iterrows()}
    print(f"Types de manifolds chargés: {len(donnees['manifolds'])} types.")
    df_trunklines = pd.read_excel(chemin_fichier, sheet_name='SpecsTrunkline')
    donnees['trunklines'] = sorted([{'diametre_in': int(r['Diametre_in']), 'cout_par_m_dollar': round(float(r['CoutParMile_dollar']) / CONVERSION_MILE_TO_M, 5), 'min_total_prod_sm3d': int(r['MinProd_Sm3d']), 'max_total_prod_sm3d': int(r['MaxProd_Sm3d'])} for i, r in df_trunklines.iterrows()], key=lambda x: x['min_total_prod_sm3d'])
    print(f"Spécifications de trunklines chargées: {len(donnees['trunklines'])} specs.")
    return donnees

def distance(p1, p2): return math.sqrt((p1.get('x', 0) - p2.get('x', 0))**2 + (p1.get('y', 0) - p2.get('y', 0))**2)
def get_trunkline_params(prod, specs):
    if not specs: return None, None
    for spec in specs:
        if spec['min_total_prod_sm3d'] <= prod <= spec['max_total_prod_sm3d']: return spec['diametre_in'], spec['cout_par_m_dollar']
    if prod > 0:
        s_specs = sorted(specs, key=lambda x: x['max_total_prod_sm3d'])
        if not s_specs: return None, None
        if prod > s_specs[-1]['max_total_prod_sm3d']: return s_specs[-1]['diametre_in'], s_specs[-1]['cout_par_m_dollar']
        if prod < s_specs[0]['min_total_prod_sm3d']: return s_specs[0]['diametre_in'], s_specs[0]['cout_par_m_dollar']
    return None, None
def get_zone_etude(puits):
    if not puits: return 0, 0, 0, 0
    all_x, all_y = [d['x'] for d in puits.values()], [d['y'] for d in puits.values()]
    return min(all_x), max(all_x), min(all_y), max(all_y)
def generer_sites_candidats_grille(x_min, x_max, y_min, y_max, pas, marge):
    sites = []
    for x in np.arange(x_min - marge, x_max + marge + pas / 2, pas):
        for y in np.arange(y_min - marge, y_max + marge + pas / 2, pas):
            sites.append({'x': round(x, 2), 'y': round(y, 2)})
    return sites
def calculer_cout_total_complet(sol, puits, m_types, t_specs, cpf_cost):
    if not sol or not sol.get('manifolds_ouverts') or not sol.get('cpf_location'): return float('inf')
    cost = cpf_cost
    active = {k: v for k, v in sol['manifolds_ouverts'].items() if v.get('puits_connectes')}
    for info in active.values(): cost += m_types[info['type_id']]['cout_installation_dollar']
    for p_id, m_id in sol['affectations_puits'].items():
        if m_id not in active: return float('inf')
        cost += distance(puits[p_id], active[m_id]) * puits[p_id]['CDF_i_dollar_par_m']
    cpf_loc = sol['cpf_location']
    for m_id, info in active.items():
        prod = sum(puits[p_id]['production_sm3d'] for p_id in info['puits_connectes'])
        d, c = get_trunkline_params(prod, t_specs)
        sol['manifolds_ouverts'][m_id]['total_production_sm3d'] = prod
        sol['manifolds_ouverts'][m_id]['DTj_in'] = d
        sol['manifolds_ouverts'][m_id]['CDTj_dollar_par_m'] = c
        if c: cost += distance(info, cpf_loc) * c
        elif prod > 0: return float('inf')
    return cost
def calculer_cout_phase1(sol, puits, m_types):
    if not sol or not sol.get('manifolds_ouverts'): return float('inf')
    cost = 0
    active = {k: v for k, v in sol['manifolds_ouverts'].items() if v.get('puits_connectes')}
    for info in active.values():
        if info['type_id'] not in m_types: return float('inf')
        cost += m_types[info['type_id']]['cout_installation_dollar']
    for p_id, m_id in sol['affectations_puits'].items():
        if m_id not in active: return float('inf')
        cost += distance(puits[p_id], active[m_id]) * puits[p_id]['CDF_i_dollar_par_m']
    return cost

# =============================================================================
# --- FONCTION DE CONSTRUCTION "CLUSTERING" (AVEC ID SIMPLIFIÉ) ---
# =============================================================================
def construire_solution_clustering(puits_data, types_manifold_data):
    clusters = {p_id: {'puits': {p_id}, 'center': p_data} for p_id, p_data in puits_data.items()}
    max_cap = max(info['capacite_puits'] for info in types_manifold_data.values())
    while True:
        if len(clusters) <= 1: break
        meilleure_fusion = {'dist': float('inf'), 'pair': None}
        cluster_ids = list(clusters.keys())
        for i in range(len(cluster_ids)):
            for j in range(i + 1, len(cluster_ids)):
                id1, id2 = cluster_ids[i], cluster_ids[j]
                c1, c2 = clusters[id1], clusters[id2]
                if len(c1['puits']) + len(c2['puits']) <= max_cap:
                    dist_clusters = distance(c1['center'], c2['center'])
                    if dist_clusters < meilleure_fusion['dist']:
                        meilleure_fusion = {'dist': dist_clusters, 'pair': (id1, id2)}
        if meilleure_fusion['pair'] is None: break
        id1, id2 = meilleure_fusion['pair']
        c1, c2 = clusters[id1], clusters[id2]
        nouveau_cluster_puits = c1['puits'].union(c2['puits'])
        nouveau_centre_x = sum(puits_data[p]['x'] for p in nouveau_cluster_puits) / len(nouveau_cluster_puits)
        nouveau_centre_y = sum(puits_data[p]['y'] for p in nouveau_cluster_puits) / len(nouveau_cluster_puits)
        nouveau_cluster = {'puits': nouveau_cluster_puits, 'center': {'x': nouveau_centre_x, 'y': nouveau_centre_y}}
        nouveau_id = f"{id1}-{id2}" # Garder une trace interne, mais on ne l'utilisera pas pour l'affichage
        del clusters[id1], clusters[id2]
        clusters[nouveau_id] = nouveau_cluster

    solution = {'manifolds_ouverts': {}, 'affectations_puits': {}}
    types_tries = sorted(types_manifold_data.items(), key=lambda item: item[1]['capacite_puits'])
    manifold_id_counter = 1
    for cluster in clusters.values():
        taille_cluster = len(cluster['puits'])
        type_choisi_id = next((tid for tid, tinfo in types_tries if taille_cluster <= tinfo['capacite_puits']), None)
        if type_choisi_id is None: return {'cout_total_phase1': float('inf')} 
        
        # Utilisation d'un ID numérique simple
        manifold_id = f"Manifold_{manifold_id_counter}"; manifold_id_counter += 1
        
        solution['manifolds_ouverts'][manifold_id] = {'type_id': type_choisi_id, 'x': cluster['center']['x'], 'y': cluster['center']['y'], 'puits_connectes': cluster['puits']}
        for p_id in cluster['puits']:
            solution['affectations_puits'][p_id] = manifold_id
    solution['cout_total_phase1'] = calculer_cout_phase1(solution, puits_data, types_manifold_data)
    return solution

# --- BOUCLE PRINCIPALE DE L'HEURISTIQUE ---
def trouver_meilleure_config_manifolds(puits_data, manifold_types):
    print(f"--- Lancement de l'Heuristique de 'Clustering Itératif' ---")
    solution_p1 = construire_solution_clustering(puits_data, manifold_types)
    cout_p1 = solution_p1.get('cout_total_phase1', float('inf'))
    if cout_p1 == float('inf'):
         print("\n--- AVERTISSEMENT: La construction de la solution a échoué. ---")
    return solution_p1, cout_p1
    
# --- OPTIMISATION CPF ---
def optimiser_cpf_pour_manifolds_fixes(solution_p1, puits_data, types_manifold_data, sites_candidats_cpf, trunkline_specs_data, cpf_cost):
    if not solution_p1 or not solution_p1.get('manifolds_ouverts'): return None, float('inf')
    meilleure_solution, meilleur_cout = None, float('inf')
    active_manifolds = {k: v for k, v in solution_p1['manifolds_ouverts'].items() if v.get('puits_connectes')}
    if not active_manifolds: return None, float('inf')
    solution_base = copy.deepcopy(solution_p1)
    solution_base['manifolds_ouverts'] = active_manifolds
    manifold_coords = {(round(m['x'],2), round(m['y'],2)) for m in active_manifolds.values()}
    for cpf_coords in sites_candidats_cpf:
        if (round(cpf_coords['x'],2), round(cpf_coords['y'],2)) in manifold_coords: continue
        solution_temp = copy.deepcopy(solution_base)
        solution_temp['cpf_location'] = cpf_coords
        cout_actuel = calculer_cout_total_complet(solution_temp, puits_data, types_manifold_data, trunkline_specs_data, cpf_cost)
        if cout_actuel < meilleur_cout:
            meilleur_cout, meilleure_solution = cout_actuel, copy.deepcopy(solution_temp)
            meilleure_solution['cout_total'] = meilleur_cout
    return meilleure_solution, meilleur_cout

# --- FONCTION D'AFFICHAGE DÉTAILLÉ ---
def afficher_resultats_detailles(solution_finale, types_manifold_data, scenario_titre=""):
    print(f"\n\n--- RÉSULTAT FINAL DÉTAILLÉ ({scenario_titre}) ---")
    
    cout_total = solution_finale.get('cout_total', 0)
    cpf_loc = solution_finale.get('cpf_location', {'x': 'N/A', 'y': 'N/A'})
    print(f"Coût Total Optimal Estimé: ${cout_total:,.2f}")
    print(f"Localisation CPF: ({cpf_loc['x']:.2f}, {cpf_loc['y']:.2f})")
    
    manifolds_actifs = solution_finale.get('manifolds_ouverts', {})
    print(f"\nNombre de Manifolds Actifs: {len(manifolds_actifs)}")
    print("-" * 50)

    for m_id, m_data in sorted(manifolds_actifs.items()):
        type_id = m_data.get('type_id', 'N/A')
        capacite = types_manifold_data.get(type_id, {}).get('capacite_puits', 'N/A')
        
        print(f"  Manifold ID: {m_id}")
        print(f"    - Coordonnées: ({m_data.get('x', 0):.2f}, {m_data.get('y', 0):.2f})")
        print(f"    - Type: {type_id} (Capacité: {capacite})")
        
        diam_trunk = m_data.get('DTj_in', 'N/A')
        print(f"    - Trunkline vers CPF: Diamètre {diam_trunk} in")
        
        puits_connectes = sorted(list(m_data.get('puits_connectes', [])))
        print(f"    - Puits Connectés ({len(puits_connectes)}):")
        
        if puits_connectes:
            for i in range(0, len(puits_connectes), 6):
                print(f"      {'  '.join(puits_connectes[i:i+6])}")
        else:
            print("      Aucun")
        
        print("-" * 50)

# =============================================================================
# --- FONCTIONS DE SCÉNARIO ---
# =============================================================================
def lancer_scenario_cpf_optimise(donnees, params):
    """Scénario 2: L'algorithme trouve le meilleur emplacement pour le CPF."""
    print("\n" + "="*60)
    print("--- SCÉNARIO 2 : CPF AVEC EMPLACEMENT OPTIMISÉ ---")
    print("="*60)

    PUITS_DATA = donnees['puits']
    TYPES_MANIFOLD_DATA = donnees['manifolds']
    TRUNKLINE_SPECS_DATA = donnees['trunklines']

    # Phase 1: Trouver la meilleure configuration de manifolds
    solution_p1, cout_p1 = trouver_meilleure_config_manifolds(PUITS_DATA, TYPES_MANIFOLD_DATA)
    
    if solution_p1 and cout_p1 != float('inf'):
        print(f"\n--- FIN PHASE 1 ---\nCoût (MFD+FL): ${cout_p1:,.2f}, Manifolds trouvés: {len(solution_p1['manifolds_ouverts'])}")
        
        # Générer les sites candidats pour le CPF
        xmin, xmax, ymin, ymax = get_zone_etude(PUITS_DATA)
        sites_candidats_cpf = generer_sites_candidats_grille(xmin, xmax, ymin, ymax, params['PAS_GRILLE_MANIFOLD_M_VAL'], params['MARGE_POUR_GRILLE_M_VAL'])
        print(f"Nombre de sites candidats pour CPF générés: {len(sites_candidats_cpf)}")
        
        # Phase 2: Optimiser l'emplacement du CPF
        print(f"\n--- DÉBUT PHASE 2: Optimisation du CPF ---")
        solution_finale, cout_final = optimiser_cpf_pour_manifolds_fixes(solution_p1, PUITS_DATA, TYPES_MANIFOLD_DATA, sites_candidats_cpf, TRUNKLINE_SPECS_DATA, params['CPF_INSTALLATION_COST_DOLLAR_VAL'])
        
        if solution_finale:
            afficher_resultats_detailles(solution_finale, TYPES_MANIFOLD_DATA, "CPF Optimisé")
        else:
            print("\nERREUR: Phase 2 n'a pas pu trouver une localisation de CPF valide.")
    else:
        print("\n--- ÉCHEC DE LA PHASE 1 ---")

def lancer_scenario_cpf_fixe(donnees, params):
    """Scénario 1: L'algorithme utilise un emplacement de CPF prédéfini."""
    print("\n" + "="*60)
    print("--- SCÉNARIO 1 : CPF AVEC EMPLACEMENT FIXE ---")
    print("="*60)
    
    PUITS_DATA = donnees['puits']
    TYPES_MANIFOLD_DATA = donnees['manifolds']
    TRUNKLINE_SPECS_DATA = donnees['trunklines']

    print(f"Utilisation de l'emplacement CPF fixe: ({params['CPF_FIXE_X']}, {params['CPF_FIXE_Y']})")

    # Phase 1: Trouver la meilleure configuration de manifolds
    solution_p1, cout_p1 = trouver_meilleure_config_manifolds(PUITS_DATA, TYPES_MANIFOLD_DATA)
    
    if solution_p1 and cout_p1 != float('inf'):
        print(f"\n--- FIN PHASE 1 ---\nCoût (Manifolds + Flowlines): ${cout_p1:,.2f}, Manifolds trouvés: {len(solution_p1['manifolds_ouverts'])}")
        
        # Calcul Final: Appliquer le coût des trunklines vers le CPF fixe
        solution_finale = copy.deepcopy(solution_p1)
        solution_finale['cpf_location'] = {'x': params['CPF_FIXE_X'], 'y': params['CPF_FIXE_Y']}
        
        cout_total_final = calculer_cout_total_complet(solution_finale, PUITS_DATA, TYPES_MANIFOLD_DATA, TRUNKLINE_SPECS_DATA, params['CPF_INSTALLATION_COST_DOLLAR_VAL'])
        
        if cout_total_final != float('inf'):
            solution_finale['cout_total'] = cout_total_final
            afficher_resultats_detailles(solution_finale, TYPES_MANIFOLD_DATA, "CPF Fixe")
        else:
            print("\nERREUR: Impossible de calculer le coût total avec le CPF fixe.")
    else:
        print("\n--- ÉCHEC DE LA PHASE 1 ---")


# =============================================================================
# --- SECTION PRINCIPALE D'EXÉCUTION ---
# =============================================================================
def main():
    try:
        fichier_excel = r'C:\Users\Dell\Downloads\donnees_probleme.xlsx'
        donnees = charger_donnees_depuis_excel(fichier_excel)
        
        # --- PARAMÈTRES COMMUNS AUX DEUX SCÉNARIOS ---
        PARAMS = {
            'CPF_INSTALLATION_COST_DOLLAR_VAL': 450000000.00,
            'MARGE_POUR_GRILLE_M_VAL': 5000.0,
            'PAS_GRILLE_MANIFOLD_M_VAL': 2000.0,
            
            # Coordonnées pour le scénario 2 (CPF Fixe)
            'CPF_FIXE_X': 449397.35,
            'CPF_FIXE_Y': 3048477.39,
        }
        
        # --- CHOISISSEZ QUEL SCÉNARIO EXÉCUTER ---
        # Décommentez la ligne du scénario que vous voulez lancer.
        
        lancer_scenario_cpf_fixe(donnees, PARAMS)
        lancer_scenario_cpf_optimise(donnees, PARAMS)
        

    except FileNotFoundError:
        print(f"\nERREUR CRITIQUE: Le fichier '{fichier_excel}' n'a pas été trouvé.")
    except Exception as e:
        print(f"\nUNE ERREUR EST SURVENUE: {e}")

if __name__ == "__main__":
    main()

--- Chargement des données depuis 'C:\Users\Dell\Downloads\donnees_probleme.xlsx' ---
Données des puits chargées: 143 puits.
Types de manifolds chargés: 3 types.
Spécifications de trunklines chargées: 13 specs.

--- SCÉNARIO 1 : CPF AVEC EMPLACEMENT FIXE ---
Utilisation de l'emplacement CPF fixe: (449397.35, 3048477.39)
--- Lancement de l'Heuristique de 'Clustering Itératif' ---

--- FIN PHASE 1 ---
Coût (Manifolds + Flowlines): $80,800,393.10, Manifolds trouvés: 11


--- RÉSULTAT FINAL DÉTAILLÉ (CPF Fixe) ---
Coût Total Optimal Estimé: $531,357,652.83
Localisation CPF: (449397.35, 3048477.39)

Nombre de Manifolds Actifs: 11
--------------------------------------------------
  Manifold ID: Manifold_1
    - Coordonnées: (473560.81, 2935659.68)
    - Type: MFD_Cap15 (Capacité: 15)
    - Trunkline vers CPF: Diamètre 10 in
    - Puits Connectés (14):
      chmp2-10  chmp2-11  chmp2-12  chmp2-13  chmp2-14  chmp2-15
      chmp2-3  chmp2-4  chmp2-W11  chmp2-W4  chmp2-W6  chmp2-W7
      chmp2-